In [5]:
import os
import boto3
import json
import pandas

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth


model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"

index_name =  os.environ.get('AWS_OPENSEARCH_INDEX_NAME')
service = 'aoss'
host = os.getenv('AWS_OPENSEARCH_ENDPOINT')
region = os.getenv('AWS_REGION')


df = pandas.read_parquet('../data/bger-2024-3-text.parquet')


# Notes for porting to AWS Serverless

Recommendation: Start with AWS Lambda
For your use case (encoding a single string with distiluse-base-multilingual-cased-v2), AWS Lambda is the better choice because:
It’s more cost-effective for low-to-moderate traffic.

The model size (135M parameters) and task (encoding a single string) fit within Lambda’s memory and timeout constraints.

Using a container image and ONNX optimization simplifies deployment and reduces latency.

Steps to Get Started:
Convert the model to ONNX to reduce size and inference time.

Build a Docker image with the model and sentence-transformers.

Deploy to Lambda with 1024-2048 MB memory and a 60-second timeout.

Test with a sample invocation and monitor cold start performance.

If latency is an issue, consider SageMaker Serverless or Provisioned Concurrency.

If you encounter issues (e.g., package size or library conflicts), refer to the Stack Overflow discussion on Lambda deployment errors and ensure a CPU-only PyTorch build.



In [6]:
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

sts = session.client("sts")
identity = sts.get_caller_identity()
print("Running as IAM identity:", identity["Arn"])

aoss_client = OpenSearch(
    hosts=[{'host': host, 'port': 443}],
    http_compress=True,
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
    timeout=36000,  # Set request timeout in seconds
)

# Check if index exists
try:
    exists = aoss_client.indices.exists(index=index_name)
    print(f"Index {index_name} exists: {exists}")
    if not exists:
        raise {"statusCode": 500, "body": json.dumps(f"Index {index_name} does not exist")}
except Exception as e:
    print(f"Error checking index: {str(e)}")
    raise {"statusCode": 500, "body": json.dumps(f"Error checking index: {str(e)}")}

Running as IAM identity: arn:aws:iam::211125557955:user/Christophe
Index fed-court-chunks-index exists: True


In [7]:

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  }
]


In [18]:
from pathlib import Path
import onnxruntime as ort
from transformers import AutoTokenizer

# Load tokenizer
onnx_path = Path("../data/onnx_model").resolve()
tokenizer = AutoTokenizer.from_pretrained(str(onnx_path))

# Load ONNX model directly
session = ort.InferenceSession(str(onnx_path / "model.onnx"))

# Prepare inputs
case_number = 6
sentence = test_cases[case_number]["content"]
inputs = tokenizer(sentence, return_tensors="np")

# Rename input keys if needed (depends on your export)
onnx_inputs = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"]
}

# Run inference
outputs = session.run(None, onnx_inputs)[0]
sentence_embedding = outputs[0]
# Use embedding for search
top_matches = 10
response = aoss_client.search(
    index=index_name,
    body={
        "size": top_matches,
        "query": {
            "knn": {
                "embedding": {
                    "vector": sentence_embedding.tolist(),
                    "k": top_matches
                }
            }
        }
    },
    timeout=30
)

hit_ids = set()
for hit in sorted(response["hits"]["hits"], key=lambda x: x["_source"]["doc_id"], reverse=True):
    hit_ids.add(hit["_source"]["doc_id"])
    print(hit["_source"]["doc_id"], hit["_score"])

relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

6B_78/2008 0.7030573
6B_642/2022 0.68748724
6B_53/2010 0.6795151
6B_475/2012 0.6890329
6B_413/2012 0.6796523
4A_642/2009 0.6779125
1B_470/2022 0.675261
1B_237/2010 0.6815866
1B_169/2016 0.68031
1B_104/2016 0.678968


[{'doc': 'Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal {T 0/2} 6B_78/2008, 6B_81/2008, 6B_90/2008 /biz Sentenza del 14 ottobre 2008 Corte di diritto penale Composizione Giudici federali Schneider, presidente, Ferrari, Favre, cancelliera Ortolano. Parti 6B_78/2008 A.________, ricorrente, patrocinato dall\'avv. Alexander Henauer, 6B_81/2008 B.________, ricorrente, patrocinato dall\'avv. Andrea Carri, 6B_90/2008 C.________, ricorrente, patrocinato dall\'avv. Pascal Cattaneo, contro Ministero pubblico del Cantone Ticino, Palazzo di giustizia, via Pretorio 16, 6901 Lugano, opponente. Oggetto Arbitrio, commisurazione della pena, sincero pentimento, ricorsi in materia penale contro la sentenza emanata il 14 dicembre 2007 dalla Corte di cassazione e di revisione penale del Tribunale d\'appello del Cantone Ticino. Fatti: A. La sera del 6 marzo 2006 A.________, B.________, C.________ e un minorenne non identificato (tale D.________) partivano da Torino in direzione della Sv

In [19]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Décision {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)


Sur la base des décisions fédérales fournies, voici un avis juridique concernant la situation d’un ressortissant érythréen débouté de sa demande d’asile, faisant recours contre sa mise en détention en vue du renvoi, en invoquant l’impossibilité de son renvoi et une violation de l’article 5 de la Convention européenne des droits de l’homme (CEDH) ainsi que de la jurisprudence fédérale sur la proportionnalité.

1. **Fondement légal de la détention en vue du renvoi**

La détention en vue du renvoi est une mesure privative de liberté justifiée par la nécessité d’assurer l’exécution effective d’une décision d’éloignement. Elle est encadrée par le droit suisse et doit respecter les garanties constitutionnelles et conventionnelles, notamment l’article 5 CEDH relatif au droit à la liberté et à la sûreté.

2. **Conditions de la détention en vue du renvoi**

Selon la jurisprudence du Tribunal fédéral (cf. notamment 1B_104/2016), la détention en vue du renvoi est admissible si :

- Le renvoi est 

In [20]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
print("Embedding dimension:", model.get_sentence_embedding_dimension())


Embedding dimension: 512
